In [3]:
import pandas as pd
import requests as rq
import time
import os
import dotenv
import re # for matching filenames to URLs
from selenium import webdriver # for retrieving DSSTOX ID and Year info from webpages
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# lost my working directory somehow
os.getcwd()
os.chdir(r'L:\\PRIV\ToxValDB\Document Webscrape\iris')

# load in the iris source file and clowder log
log = pd.read_csv(r'pyclowderext/clowder_log_pyclowderext_iris_20220727.csv')
iris = pd.read_excel('iris_assessment_table_unique_names_20220721.xlsx')

# Clowder API key
load_dotenv()
key = os.getenv('apiKey')

* While you're working, keep in mind that the first three rows of the log are for the dataset and its two subdirectories.
* While some share dates, in many cases, each of the several tables on each webpage has its own "last updated" date, making retrieval of one year to add to the Clowder metadata for that document fraught. I've left it blank for now.

In [4]:
# need to prepend three dummy values to make the rows line up
filenames = ["NA", "NA", "NA"]

# pick a Clowder ID, grab its metadata, extract the filename, and add it to the list
for i in range(3, len(log)):
    uuid = log.loc[i, 'uuid']
    name = rq.get('https://clowder.edap-cluster.com/api/files/'+str(uuid)+'/metadata', 
                headers = {'X-API-Key': key}).json()['filename']
    time.sleep(0.25)
    filenames.append(name)
    if i % 10 == 0:
        print(i/len(log)*100)

log['File Name'] = filenames

0.5824111822947
1.1648223645894
1.7472335468841003
2.3296447291788
2.9120559114735003
3.4944670937682005
4.0768782760629
4.6592894583576
5.241700640652301
5.824111822947001
6.4065230052417
6.988934187536401
7.571345369831101
8.1537565521258
8.736167734420501
9.3185789167152
9.900990099009901
10.483401281304602
11.0658124635993
11.648223645894001
12.2306348281887
12.8130460104834
13.395457192778101
13.977868375072802
14.560279557367501
15.142690739662202
15.725101921956902
16.3075131042516
16.8899242865463
17.472335468841003
18.0547466511357
18.6371578334304
19.219569015725103
19.801980198019802
20.3843913803145
20.966802562609203
21.549213744903902
22.1316249271986
22.714036109493303
23.296447291788002
23.878858474082705
24.4612696563774
25.0436808386721
25.6260920209668
26.2085032032615
26.790914385556203
27.373325567850905
27.955736750145604
28.538147932440307
29.120559114735002
29.7029702970297
30.285381479324403
30.867792661619102
31.450203843913805
32.032615026208504
32.6150262085

In [5]:
Options = webdriver.ChromeOptions()
Options.headless = True 
path=Service(r'C:\Users\JWILLI29\OneDrive - Environmental Protection Agency (EPA)\Profile\Documents\chromedriver.exe')
driver = webdriver.Chrome(service=path, options = Options)

for i in range(len(iris)):
    # open up the webpage
    url = iris.loc[i, 'Chemical URL']
    driver.get(url)
    time.sleep(0.25)
    
    # grab the CAS number and DTXS ID, add the latter to the list
    # we have to try/except this because it only works for the webpages, not the screenshots or summaries
    # oh get the CAS numbers too, why not. you never know when you might need 'em (e.g. for document mapping)
    try:
        chem_ids = driver.find_element(By.XPATH, "//*[@id=\"main-content\"]/div[2]/div/div/div/div[1]/h3").text
        iris.loc[i, 'CASRN'] = chem_ids.split(' | ')[0]
        iris.loc[i, 'DSSTox Structure ID'] = chem_ids.split(' | ')[1]
        time.sleep(0.25)
        print(round(i/len(iris) * 100, 2))
    except:
        print('error at row #'+i)
    
driver.quit()

iris.to_excel('iris_temp2.xlsx', index = False)

0.0
0.18
0.35
0.53
0.7
0.88
1.05
1.23
1.4
1.58
1.75
1.93
2.1
2.28
2.45
2.63
2.8
2.98
3.15
3.33
3.5
3.68
3.85
4.03
4.2
4.38
4.55
4.73
4.9
5.08
5.25
5.43
5.6
5.78
5.95
6.13
6.3
6.48
6.65
6.83
7.01
7.18
7.36
7.53
7.71
7.88
8.06
8.23
8.41
8.58
8.76
8.93
9.11
9.28
9.46
9.63
9.81
9.98
10.16
10.33
10.51
10.68
10.86
11.03
11.21
11.38
11.56
11.73
11.91
12.08
12.26
12.43
12.61
12.78
12.96
13.13
13.31
13.49
13.66
13.84
14.01
14.19
14.36
14.54
14.71
14.89
15.06
15.24
15.41
15.59
15.76
15.94
16.11
16.29
16.46
16.64
16.81
16.99
17.16
17.34
17.51
17.69
17.86
18.04
18.21
18.39
18.56
18.74
18.91
19.09
19.26
19.44
19.61
19.79
19.96
20.14
20.32
20.49
20.67
20.84
21.02
21.19
21.37
21.54
21.72
21.89
22.07
22.24
22.42
22.59
22.77
22.94
23.12
23.29
23.47
23.64
23.82
23.99
24.17
24.34
24.52
24.69
24.87
25.04
25.22
25.39
25.57
25.74
25.92
26.09
26.27
26.44
26.62
26.8
26.97
27.15
27.32
27.5
27.67
27.85
28.02
28.2
28.37
28.55
28.72
28.9
29.07
29.25
29.42
29.6
29.77
29.95
30.12
30.3
30.47
30.65
30.82
31.0
31.17
3

In [6]:
# use the chemical names to match URLs and DSSTox IDs from iris
urls = ["NA", "NA", "NA"]
dsstoxids = ["NA", "NA", "NA"]
casrns = ['NA', 'NA', 'NA']
names = iris['Chemical Name'].tolist()

for s in filenames[3:]:
    # ignore the Thumbs.db file
    if bool(re.fullmatch(r'Thumbs.db', s)) == True:
        urls.append("NA")
        dsstoxids.append("NA")
        casrns.append('NA')
        continue
    
    # check to see which type of file we're dealing with
    # since the pdfs have identical naming conventions, we slice them the same way and combine those checks
    if bool(re.fullmatch(r'.*.pdf', s)) == True:
        # extract and find the chemical name in iris
        index = names.index(s[5:-28])
        # add the corresponding URL to our list
        if bool(re.search('_summary', s)):
            urls.append(iris.loc[index, 'IRIS Summary URL'])
        else:
            urls.append(iris.loc[index, 'Chemical URL'])
        # and add the corresponding DSSTOX ID and CASRN to our other lists
        dsstoxids.append(iris.loc[index, 'DSSTox Structure ID'])
        casrns.append(iris.loc[index, 'CASRN'])
    else:
        index = names.index(s[:-4])
        urls.append(iris.loc[index, 'Chemical URL'])
        dsstoxids.append(iris.loc[index, 'DSSTox Structure ID'])
        casrns.append(iris.loc[index, 'CASRN'])
            
log['DSSTox Structure ID'] = dsstoxids
log['CASRN'] = casrns
log.to_excel('temp_log2.xlsx')

In [7]:
log['Source'] = "EPA IRIS"
log['Subsource'] = "EPA IRIS"
log['Author'] = "U.S. Environmental Protection Agency (EPA)"

In [8]:
metadata = log[['uuid', 'File Name', 'URL', 'DOI', 'HERO ID', 'Source', 'Subsource', 'Author', 'Year', 'Study Name', 'DSSTox Structure ID', 'CASRN']]
metadata[3:].to_excel('metadata2.xlsx', index = False)